In [13]:
import cv2
from ultralytics import YOLO
from PIL import Image

In [14]:
img_path = "./mujoco_objects.png"
frame = cv2.imread(img_path)
if frame is None:
    raise FileNotFoundError(f"이미지를 찾을 수 없습니다: {img_path}")

frame = cv2.resize(frame, (640, 480))
model = YOLO("yolo11n.pt")
results = model(source=frame, conf=0.5, verbose=True)


0: 480x640 1 can, 1 milk, 1 lemon, 150.7ms
Speed: 4.3ms preprocess, 150.7ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)


In [16]:
for i, r in enumerate(results):
    im_bgr = r.plot()  
    im_rgb = Image.fromarray(im_bgr[..., ::-1]) 
    r.show()

In [19]:
import numpy as np
import mujoco as mj
from mujoco.glfw import glfw # MuJoCo 시뮬레이터 GLFW (OpenGL 기반 윈도우/입력 라이브러리)

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from utils.mouse_callbacks import MouseCallbacks # 마우스 이벤트(카메라 회전/이동/줌 등) 콜백
from utils.keyboard_callbacks import KeyboardCallbacks # 키보드 입력(로봇 제어 및 초기화) 콜백 
from utils.scene_creator import SceneCreator # MJCF xml 파일 조합 및 scene 빌드용 헬퍼

In [20]:
cwd = os.getcwd() 
PROJECT_ROOT = os.path.dirname(cwd)
print(f"project root: {PROJECT_ROOT}")

assets_dir = os.path.join(PROJECT_ROOT, "assets")
base_env_path = os.path.join(assets_dir, "scenes", "floor_sky.xml")   # 환경 MJCF
robot_path = os.path.join(assets_dir, "robots", "pinky", "pinky.xml") # 로봇 MJCF

# 배치할 오브젝트 정보
objects_to_spawn = [
    {"path": os.path.join(assets_dir, "objects", "can.xml"), "name": "can_1",  "pos": "0.5 0.3 0.05"},
    {"path": os.path.join(assets_dir, "objects", "milk.xml"), "name": "milk_1", "pos": "0.5 0.0 0.05"},
    {"path": os.path.join(assets_dir, "objects", "lemon.xml"), "name": "lemon_1","pos": "0.5 -0.3 0.05"},
]

# 각 MJCF들을 병합
scene_xml_string = SceneCreator.build_mjcf_scene(
    base_env_path=base_env_path,
    robot_path=robot_path,
    objects_to_spawn=objects_to_spawn,
    save_xml=False
)

project root: c:\Users\addinedu\short\pinky_mujoco_menagerie


In [21]:
# XML 문자열로부터 MuJoCo 모델(물리/구조 정보) 생성
model = mj.MjModel.from_xml_string(scene_xml_string)
# 모델에 대한 시뮬레이션 데이터(상태 변수, 센서값 등) 생성
data = mj.MjData(model)

# mujoco 카메라(메인, 로봇 시점), 옵션, 씬(각 시점의 3D 그래픽 정보) 객체 선언
main_cam = mj.MjvCamera()
robot_cam = mj.MjvCamera()
opt = mj.MjvOption()
scene_main = mj.MjvScene(model, maxgeom=10000)
scene_robot = mj.MjvScene(model, maxgeom=10000)

In [22]:
# GLFW 초기화 및 윈도우 생성
glfw.init()
window_main = glfw.create_window(900, 900, "Main View", None, None)
window_robot = glfw.create_window(640, 480, "Camera View", None, window_main)

# OpenGL 컨텍스트 연결 및 렌더 컨텍스트 생성
glfw.make_context_current(window_main)
ctx_main = mj.MjrContext(model, mj.mjtFontScale.mjFONTSCALE_150.value)
glfw.make_context_current(window_robot)
ctx_robot = mj.MjrContext(model, mj.mjtFontScale.mjFONTSCALE_150.value)

# 수직 동기화 설정: 1 활성화, 0 비활성화
glfw.swap_interval(1)
# Free Camera, 렌더링 옵션 기본값
mj.mjv_defaultFreeCamera(model, main_cam)
mj.mjv_defaultOption(opt)

In [23]:
# 메인 카메라: 자유 이동(Free) 모드 설정
main_cam.type = mj.mjtCamera.mjCAMERA_FREE
main_cam.azimuth = 0     # 카메라 수평(좌우) 회전 각도
main_cam.elevation = -30 # 카메라 수직(상하) 회전 각도
main_cam.distance = 2    # 카메라와 대상(lookat) 사이 거리
main_cam.lookat = np.array([0.0, 0.0, 0.5]) # 카메라가 바라보는 좌표

# 로봇 카메라: 고정(Fixed) 모드 및 특정 카메라 ID로 지정
robot_cam.type = mj.mjtCamera.mjCAMERA_FIXED 
cam_id = mj.mj_name2id(model, mj.mjtObj.mjOBJ_CAMERA, "camera")
robot_cam.fixedcamid = cam_id

# 메인 씬: 그림자/반사 효과 비활성화 (성능 향상 목적)
scene_main.flags[mj.mjtRndFlag.mjRND_SHADOW] = False
scene_main.flags[mj.mjtRndFlag.mjRND_REFLECTION] = False

In [24]:
# 수직 동기화 설정: 1 활성화, 0 비활성화
glfw.swap_interval(1)

# 콜백 핸들러 객체 생성
mousecallbacks = MouseCallbacks()
kbdcallbacks = KeyboardCallbacks()

# GLFW 콜백 등록 (키보드, 커서, 마우스 버튼, 마우스 스크롤)
glfw.set_key_callback(window_main, 
                    lambda w, k, sc, act, m: kbdcallbacks.keyboardGLFW(w, k, sc, act, m, model, data))
glfw.set_cursor_pos_callback(window_main, 
                    lambda w, x, y: mousecallbacks.mouse_move(w, x, y, model, scene_main, main_cam))
glfw.set_mouse_button_callback(window_main, 
                    lambda w, b, act, m: mousecallbacks.mouse_button(w, b, act, m))
glfw.set_scroll_callback(window_main, 
                    lambda w, xo, yo: mousecallbacks.scroll(w, xo, yo, model, scene_main, main_cam))

In [25]:
from utils.mujoco_renderer import Renderer
from utils.camera_recorder import CameraRecorder

main_renderer = Renderer(window_main, scene_main, ctx_main, main_cam, model, data, opt)
robot_renderer = Renderer(window_robot, scene_robot, ctx_robot, robot_cam, model, data, opt)
robot_view_width, robot_view_height = glfw.get_framebuffer_size(window_robot)

output_path = "record.mp4"

recorder = CameraRecorder(
    window=window_robot,
    width=robot_view_width, 
    height=robot_view_height, 
    filename=output_path,
)

In [26]:
while not glfw.window_should_close(window_main):
    time_prev = data.time
    while (data.time - time_prev < 1.0 / 60.0): # 60fps
        try:
            mj.mj_step(model, data)
        except mj.MjError as e:
            print(f"시뮬레이션 불안정성 감지: {e}. 데이터를 리셋합니다.")
            mj.mj_resetData(model, data)
            mj.mj_forward(model, data)
            break

    main_renderer.render_window(overlay_type="robot_pose")
    robot_renderer.render_window(overlay_type=None)
    recorder.capture_and_write()
    glfw.poll_events()
glfw.terminate()
recorder.release()

Camera view has been recorded to record.mp4


### 데이터 셋 생성

In [17]:
from utils.video_to_frame_converter import VideoToFramesConverter

frame_saver = VideoToFramesConverter(
    video_path="./record.mp4",
    output_dir="./image_datasets/images",
    saving_fps=4
)

frame_saver.save_frames()

프레임 저장 경로 : ./image_datasets/images
비디오 프레임의 총 프레임 수 : 1733
초당 프레임 수 : 30.0
저장할 프레임의 fps : 4


Processing frames: 100%|██████████| 1733/1733 [00:01<00:00, 871.35it/s]

총 저장된 프레임 수 : 248
